In [23]:
%load_ext autoreload
%autoreload 2

import json

from grants_tagger.slim.mesh_xlinear import train, evaluate

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Train Xlinear model (for fast iteration/experimentation/etc)

This notebook trains xlinear models for fast experimentation. To get the toy data, you need to run dvc from the root of folder:

`dvc pull -s dvc.yaml:preprocess_bioasq_mesh_toy`

Replace `train_mesh2021_toy` and `test_mesh2021_toy` without the suffix (`{train,test}_mesh2021.jsonl`) for a full experience of training (see training times in `results`)

In [41]:
parameters = {
    'ngram_range': (1, 1), 
    'beam_size': 30,
    'only_topk': 200, 
    'min_weight_value': 0.1,
    'max_features': 400_000
}

In [25]:
!curl -X POST -H 'Content-type: application/json' --data "{'text': 'Hi <$SLACK_USER>, training has started'}" $SLACK_HOOK

ok

In [29]:
model, label_binarizer = train(
    train_data_path='../data/processed/train_mesh2021_toy.jsonl',
    label_binarizer_path='../models/label_binarizer-toy.pkl',
    parameters=parameters,
    model_path='../models/xlinear-toy'
)

../models/xlinear-toy-test/label_binarizer.pkl exists. Loading existing
Loading data...
Fitting model
Saving model


In [36]:
results, full_report = evaluate(
    model,
    label_binarizer,
    train_data_path='../data/processed/train_mesh2021_toy.jsonl',
    test_data_path='../data/processed/test_mesh2021_toy.jsonl',
    results_path='../results/results_toy.json',
    full_report_path='../results/full_report_toy.json'
)

Loading data...
Loading data...
Evaluating model


/data/grants_tagger/venv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:994: UserWarning: unknown class(es) ['Chikungunya Fever', 'Chikungunya virus', 'Continuous Positive Airway Pressure', 'Data Analysis', 'Dermatitis, Occupational', 'Drug Administration Schedule', 'Lupus Erythematosus, Systemic', 'Measles', 'Measles Vaccine', 'Printing, Three-Dimensional', 'Psychometrics', "Sjogren's Syndrome", 'Skin Diseases', 'Zika Virus', 'Zika Virus Infection'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'
/data/grants_tagger/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/grants_tagger/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-scor

In [37]:
results

{'threshold': '0.50', 'precision': '0.88', 'recall': '0.45', 'f1': '0.59'}

In [39]:
full_report['Coronavirus']

{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 0}

In [40]:
!curl -X POST -H 'Content-type: application/json' --data "{'text': 'Hi <$SLACK_USER>, training has finished'}" $SLACK_HOOK

ok